<a href="https://colab.research.google.com/github/ganesh-deshmukh/Google-colab-hcr/blob/master/alphabets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!kaggle datasets download -d sachinpatel21/az-handwritten-alphabets-in-csv-format

az-handwritten-alphabets-in-csv-format.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
!unzip az-handwritten-alphabets-in-csv-format.zip

Archive:  az-handwritten-alphabets-in-csv-format.zip
  inflating: A_Z Handwritten Data.zip  


In [10]:
!unzip "A_Z Handwritten Data.zip"


Archive:  A_Z Handwritten Data.zip
  inflating: A_Z Handwritten Data.csv  


 az-handwritten-alphabets-in-csv-format.zip
'A_Z Handwritten Data.csv'
'A_Z Handwritten Data.zip'
'kaggle (1).json'
 kaggle.json
 sample_data
 six-resized.png
 stack-overflow-2018-developer-survey.zip


In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras import backend as K
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import numpy as np

# seed for reproducing same results
seed = 785
np.random.seed(seed)

# load dataset
dataset = np.loadtxt('./A_Z Handwritten Data.csv', delimiter=',')

# split into input and output variables
X = dataset[:,0:784]
Y = dataset[:,0]

# split the data into training (50%) and testing (50%)
(X_train, X_test, Y_train, Y_test) = train_test_split(X, Y, test_size=0.50, random_state=seed)

X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')

X_train = X_train / 255
X_test = X_test / 255

# one hot encode outputs
Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)

num_classes = Y_test.shape[1]


# create model
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=10, batch_size=200, verbose=2)

# Final evaluation of the model
scores = model.evaluate(X_test,Y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

model.save('weights.model')

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 186225 samples, validate on 186226 samples
Epoch 1/10
 - 182s - loss: 0.2826 - acc: 0.9222 - val_loss: 0.1059 - val_acc: 0.9711
Epoch 2/10
 - 182s - loss: 0.1016 - acc: 0.9719 - val_loss: 0.0831 - val_acc: 0.9767
Epoch 3/10
 - 181s - loss: 0.0775 - acc: 0.9781 - val_loss: 0.0643 - val_acc: 0.9823
Epoch 4/10
 - 184s - loss: 0.0629 - acc: 0.9819 - val_loss: 0.0615 - val_acc: 0.9831
Epoch 5/10
 - 181s - loss: 0.0519 - acc: 0.9849 - val_loss: 0.0561 - val_acc: 0.9847
Epoch 6/10
 - 181s - loss: 0.0426 - acc: 0.9874 - val_loss: 0.0537 - val_acc: 0.9855
Epoch 7/10
 - 181s - loss: 0.0354 - acc: 0.9894 - val_loss: 0.0534 - val_acc: 0.9855
Epoch 8/10
 - 181s - loss: 0.0308 - acc: 0.9907 - val_loss: 0.0470 - val_acc: 0.9876
Epoch 9/10
 - 182s - loss: 0.0250